In [4]:
import pandas as pd

# CSV読み込み（文字化け防止で encoding 指定）
df = pd.read_csv(
    "todos_logs_export_20251215_054532.csv",
)

# 先頭5行を表示
df.head()


,userId,todoId,taskName,date,actualMinutes,estimatedMinutes,deadline,completed
0,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,2025-10-16,20,240,2025-10-21T05:40:00.000Z,True
1,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,2025-10-18,30,240,2025-10-21T05:40:00.000Z,True
2,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,2025-10-19,90,240,2025-10-21T05:40:00.000Z,True
3,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,2025-10-20,120,240,2025-10-21T05:40:00.000Z,True
4,DgApBqtxpeVE5G5MnIVw9ThpyN93,5NHkI2KZn1oWspLbrLVE,日本学生支援機構採否通知配布,2025-10-21,60,60,2025-10-16T02:40:00.000Z,True


In [5]:
# date を datetime に
df["date"] = pd.to_datetime(df["date"])

# 数値列を明示的に数値に
df["actualMinutes"] = pd.to_numeric(df["actualMinutes"], errors="coerce").fillna(0)
df["estimatedMinutes"] = pd.to_numeric(df["estimatedMinutes"], errors="coerce")

df.dtypes


,0
userId,object
todoId,object
taskName,object
date,datetime64[ns]
actualMinutes,int64
estimatedMinutes,int64
deadline,object
completed,bool


In [6]:
# タスク単位・日付単位で 0埋め
full_rows = []

for (userId, todoId), g in df.groupby(["userId", "todoId"]):
    g = g.sort_values("date")

    date_range = pd.date_range(
        start=g["date"].min(),
        end=g["date"].max(),
        freq="D"
    )

    g_full = (
        g.set_index("date")
         .reindex(date_range)
         .assign(userId=userId, todoId=todoId)
         .reset_index()
         .rename(columns={"index": "date"})
    )

    g_full["actualMinutes"] = g_full["actualMinutes"].fillna(0)
    g_full["estimatedMinutes"] = g["estimatedMinutes"].iloc[0]
    g_full["deadline"] = g["deadline"].iloc[0]
    g_full["taskName"] = g["taskName"].iloc[0]
    g_full["completed"] = g["completed"].iloc[0]

    full_rows.append(g_full)

df_full = pd.concat(full_rows, ignore_index=True)
df_full.head()


,date,userId,todoId,taskName,actualMinutes,estimatedMinutes,deadline,completed
0,2025-10-16,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,20.0,240,2025-10-21T05:40:00.000Z,True
1,2025-10-17,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,0.0,240,2025-10-21T05:40:00.000Z,True
2,2025-10-18,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,30.0,240,2025-10-21T05:40:00.000Z,True
3,2025-10-19,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,90.0,240,2025-10-21T05:40:00.000Z,True
4,2025-10-20,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,120.0,240,2025-10-21T05:40:00.000Z,True


In [7]:
# 累積実績（タスク単位）
df_full = df_full.sort_values(["userId", "todoId", "date"])

df_full["cumulativeActual"] = (
    df_full.groupby(["userId", "todoId"])["actualMinutes"]
           .cumsum()
)

df_full.head()


,date,userId,todoId,taskName,actualMinutes,estimatedMinutes,deadline,completed,cumulativeActual
0,2025-10-16,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,20.0,240,2025-10-21T05:40:00.000Z,True,20.0
1,2025-10-17,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,0.0,240,2025-10-21T05:40:00.000Z,True,20.0
2,2025-10-18,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,30.0,240,2025-10-21T05:40:00.000Z,True,50.0
3,2025-10-19,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,90.0,240,2025-10-21T05:40:00.000Z,True,140.0
4,2025-10-20,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,中間発表の資料を完成させる,120.0,240,2025-10-21T05:40:00.000Z,True,260.0


In [9]:
# deadline を datetime に変換（UTC Z 対応）
df_full["deadline"] = pd.to_datetime(df_full["deadline"], errors="coerce")


In [11]:
import pandas as pd

# date を UTC の tz-aware にする
df_full["date"] = pd.to_datetime(df_full["date"], errors="coerce", utc=True)

# deadline も UTC の tz-aware にする（Z付きなのでこれで揃う）
df_full["deadline"] = pd.to_datetime(df_full["deadline"], errors="coerce", utc=True)


In [12]:
import numpy as np

# 残り日数（最低1日）
df_full["daysLeft"] = (
    (df_full["deadline"] - df_full["date"])
    .dt.days
    .clip(lower=1)
)

# 残り作業量
df_full["remainingWork"] = (
    df_full["estimatedMinutes"] - df_full["cumulativeActual"]
).clip(lower=0)

# 要求ペース
df_full["requiredPace"] = df_full["remainingWork"] / df_full["daysLeft"]


In [13]:
# 直近7日平均ペース
df_full["pace7d"] = (
    df_full
    .groupby(["userId", "todoId"])["actualMinutes"]
    .transform(lambda x: x.rolling(7, min_periods=1).mean())
)


In [14]:
# SPI = pace7d / requiredPace
df_full["SPI"] = np.where(
    df_full["requiredPace"] > 0,
    df_full["pace7d"] / df_full["requiredPace"],
    np.nan
)


In [16]:
df_work = df_full[df_full["remainingWork"] > 0].copy()

In [17]:
df_full[[
    "date",
    "actualMinutes",
    "pace7d",
    "requiredPace",
    "SPI"
]].head(15)


,date,actualMinutes,pace7d,requiredPace,SPI
0,2025-10-16 00:00:00+00:00,20.0,20.000000,44.000000,0.454545
1,2025-10-17 00:00:00+00:00,0.0,10.000000,55.000000,0.181818
2,2025-10-18 00:00:00+00:00,30.0,16.666667,63.333333,0.263158
3,2025-10-19 00:00:00+00:00,90.0,35.000000,50.000000,0.700000
4,2025-10-20 00:00:00+00:00,120.0,52.000000,0.000000,NaN
5,2025-10-21 00:00:00+00:00,60.0,60.000000,0.000000,NaN
6,2025-10-15 00:00:00+00:00,180.0,180.000000,0.000000,NaN
7,2025-10-21 00:00:00+00:00,30.0,30.000000,0.000000,NaN
8,2025-10-18 00:00:00+00:00,15.0,15.000000,45.000000,0.333333
9,2025-10-19 00:00:00+00:00,30.0,22.500000,15.000000,1.500000


In [18]:
TH = 0.8
events = df_work[df_work["SPI"] < TH].copy()

print("イベント件数:", len(events))
events[["userId","todoId","date","SPI","pace7d","requiredPace"]].head(10)


イベント件数: 139


,userId,todoId,date,SPI,pace7d,requiredPace
0,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-16 00:00:00+00:00,0.454545,20.000000,44.000000
1,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-17 00:00:00+00:00,0.181818,10.000000,55.000000
2,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-18 00:00:00+00:00,0.263158,16.666667,63.333333
3,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-19 00:00:00+00:00,0.700000,35.000000,50.000000
8,DgApBqtxpeVE5G5MnIVw9ThpyN93,LZcCtTroo4OcXLOZCSjS,2025-10-18 00:00:00+00:00,0.333333,15.000000,45.000000
11,DgApBqtxpeVE5G5MnIVw9ThpyN93,OBRscVWDrP6T2dgGwzRM,2025-12-09 00:00:00+00:00,0.600000,90.000000,150.000000
13,DgApBqtxpeVE5G5MnIVw9ThpyN93,aQInLuXM80dQeHCpz6vs,2025-11-25 00:00:00+00:00,0.600000,60.000000,100.000000
14,DgApBqtxpeVE5G5MnIVw9ThpyN93,aQInLuXM80dQeHCpz6vs,2025-11-26 00:00:00+00:00,0.333333,40.000000,120.000000
15,DgApBqtxpeVE5G5MnIVw9ThpyN93,aQInLuXM80dQeHCpz6vs,2025-11-27 00:00:00+00:00,0.714286,75.000000,105.000000
16,DgApBqtxpeVE5G5MnIVw9ThpyN93,aQInLuXM80dQeHCpz6vs,2025-11-28 00:00:00+00:00,0.428571,60.000000,140.000000


In [19]:
import pandas as pd
from datetime import timedelta

rows = []
for _, e in events.iterrows():
    uid = e["userId"]
    tid = e["todoId"]
    t0  = e["date"]

    g = df_work[(df_work["userId"]==uid) & (df_work["todoId"]==tid)].copy()
    g = g.sort_values("date")

    before = g[(g["date"] >= t0 - pd.Timedelta(days=7)) & (g["date"] <  t0)]
    after  = g[(g["date"] >  t0) & (g["date"] <= t0 + pd.Timedelta(days=7))]

    pace_before = before["actualMinutes"].mean()
    pace_after  = after["actualMinutes"].mean()

    rows.append({
        "userId": uid,
        "todoId": tid,
        "eventDate": t0,
        "spi_event": float(e["SPI"]),
        "pace_before7": float(pace_before) if pd.notna(pace_before) else None,
        "pace_after7":  float(pace_after)  if pd.notna(pace_after)  else None,
    })

event_table = pd.DataFrame(rows)
event_table["d_pace"] = event_table["pace_after7"] - event_table["pace_before7"]

event_table.head()


,userId,todoId,eventDate,spi_event,pace_before7,pace_after7,d_pace
0,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-16 00:00:00+00:00,0.454545,NaN,40.0,NaN
1,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-17 00:00:00+00:00,0.181818,20.000000,60.0,40.0
2,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-18 00:00:00+00:00,0.263158,10.000000,90.0,80.0
3,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-19 00:00:00+00:00,0.700000,16.666667,NaN,NaN
4,DgApBqtxpeVE5G5MnIVw9ThpyN93,LZcCtTroo4OcXLOZCSjS,2025-10-18 00:00:00+00:00,0.333333,NaN,30.0,NaN


In [20]:
event_valid = event_table.dropna(
    subset=["pace_before7", "pace_after7"]
).copy()

print("全イベント数:", len(event_table))
print("検定可能イベント数:", len(event_valid))

event_valid


全イベント数: 139
検定可能イベント数: 121


,userId,todoId,eventDate,spi_event,pace_before7,pace_after7,d_pace
1,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-17 00:00:00+00:00,0.181818,20.000000,60.0,40.000000
2,DgApBqtxpeVE5G5MnIVw9ThpyN93,53EIH4uY2PeAUhQgrVkk,2025-10-18 00:00:00+00:00,0.263158,10.000000,90.0,80.000000
6,DgApBqtxpeVE5G5MnIVw9ThpyN93,aQInLuXM80dQeHCpz6vs,2025-11-25 00:00:00+00:00,0.600000,120.000000,90.0,-30.000000
7,DgApBqtxpeVE5G5MnIVw9ThpyN93,aQInLuXM80dQeHCpz6vs,2025-11-26 00:00:00+00:00,0.333333,60.000000,108.0,48.000000
8,DgApBqtxpeVE5G5MnIVw9ThpyN93,aQInLuXM80dQeHCpz6vs,2025-11-27 00:00:00+00:00,0.714286,40.000000,90.0,50.000000
...,...,...,...,...,...,...,...
133,Z9IYVixbY7QYdNBlmHkyIUEXMAI2,qZKJCL2VufLaTX9DEQp4,2025-12-08 00:00:00+00:00,0.000000,1.428571,30.0,28.571429
134,Z9IYVixbY7QYdNBlmHkyIUEXMAI2,uQW6hsvE1slri1CR9ckl,2025-11-21 00:00:00+00:00,0.666667,30.000000,0.0,-30.000000
135,Z9IYVixbY7QYdNBlmHkyIUEXMAI2,uQW6hsvE1slri1CR9ckl,2025-11-22 00:00:00+00:00,0.250000,20.000000,0.0,-20.000000
136,Z9IYVixbY7QYdNBlmHkyIUEXMAI2,uQW6hsvE1slri1CR9ckl,2025-11-23 00:00:00+00:00,0.200000,15.000000,0.0,-15.000000


In [21]:
event_valid["d_pace"] = (
    event_valid["pace_after7"] - event_valid["pace_before7"]
)

event_valid[["eventDate","pace_before7","pace_after7","d_pace"]]


,eventDate,pace_before7,pace_after7,d_pace
1,2025-10-17 00:00:00+00:00,20.000000,60.0,40.000000
2,2025-10-18 00:00:00+00:00,10.000000,90.0,80.000000
6,2025-11-25 00:00:00+00:00,120.000000,90.0,-30.000000
7,2025-11-26 00:00:00+00:00,60.000000,108.0,48.000000
8,2025-11-27 00:00:00+00:00,40.000000,90.0,50.000000
...,...,...,...,...
133,2025-12-08 00:00:00+00:00,1.428571,30.0,28.571429
134,2025-11-21 00:00:00+00:00,30.000000,0.0,-30.000000
135,2025-11-22 00:00:00+00:00,20.000000,0.0,-20.000000
136,2025-11-23 00:00:00+00:00,15.000000,0.0,-15.000000


In [23]:
from scipy.stats import shapiro

diff = event_valid["d_pace"]

stat, p = shapiro(diff)
print("Shapiro-Wilk p-value:", p)


Shapiro-Wilk p-value: 5.759395072565013e-12


In [24]:
from scipy.stats import ttest_rel

t, p = ttest_rel(
    event_valid["pace_after7"],
    event_valid["pace_before7"]
)

print("t =", t)
print("p =", p)


t = 1.709123269940458
p = 0.0900130903631527


In [25]:
from scipy.stats import wilcoxon

w, p = wilcoxon(diff)
print("W =", w)
print("p =", p)


W = 1953.0
p = 0.839497092684152
